In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import json
import pickle
from pathlib import Path
from datetime import datetime
import warnings
import sys
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Configuration
BASE_DIR = Path().resolve().parent.parent
sys.path.append(str(BASE_DIR / "src"))

# Répertoires
DATA_DIR = BASE_DIR / "data"

SRC_PATH = BASE_DIR / "src"
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))
    
# Définition des chemins en fonction de ta structure actuelle

PROCESSED_DIR = DATA_DIR / "processed"
EXPORTS_DIR = DATA_DIR / "exports"
VISUALS_DIR = PROCESSED_DIR / "visuals"
RESULTS_DIR = DATA_DIR / "results"
RESULTS_DIR.mkdir(exist_ok=True)



In [6]:



from scripts.summarizer_engine import summarize_text_with_ensemble
from evaluation.comprehensive_evaluator import ComprehensiveEvaluator
from evaluation.human_evaluation import HumanEvaluationInterface
import pandas as pd
import json

pd.set_option('display.max_colwidth', 300)


In [ ]:
# Choisir ton fichier de calibration ou corpus (exemple : calibration_corpus_300.json)
CORPUS_PATH = PROCESSED_DIR / "calibration_corpus_300.json"

with open(CORPUS_PATH, "r", encoding="utf-8") as f:
    corpus_data = json.load(f)





{'id': 491,
 'title': 'Comment le Wi-Fi peut permettre de vous pister, même à travers les murs',
 'text_cleaned': 'Avec l’IA, il est possible de suivre à la trace une personne en se servant des ondes Wi-Fi. Des chercheurs italiens ont en effet mis au point un système de surveillance qui analyse la manière dont chaque corps humain perturbe les signaux Wi-Fi. Des chercheurs italiens ont découvert qu’il était possible de pister un individu en se servant du Wi-Fi. Développé par des scientifiques de l’Université La Sapienza de Rome, l’outil, appelé « WhoFi », est capable de reconnaître une personne rien qu’en analysant la manière dont son corps interagit avec les ondes Wi-Fi. Une empreinte invisible En effet, le corps d’une personne interfère de façon unique avec le Wi-Fi. Les interférences dépendent de la forme du corps humain en question. « L’idée centrale est que, lorsqu’un signal Wi-Fi se propage dans un environnement, sa forme d’onde est modifiée par la présence et les caractéristiques

In [ ]:
corpus_data[0]

In [9]:
def get_text(doc):
    if "text_cleaned" in doc:
        return doc["text_cleaned"]
    elif "text" in doc:
        return doc["text"]
    else:
        raise KeyError("Aucun champ 'text' ou 'text_cleaned' dans ce document.")

raw_text = get_text(corpus_data[0]) if isinstance(corpus_data, list) else get_text(corpus_data)


In [12]:
# Charger le CSV ou le PKL pour résumer un article
ARTICLES_CSV_PATH = PROCESSED_DIR / "articles_preprocessed_summary.csv"
df_articles = pd.read_csv(ARTICLES_CSV_PATH)
print(df_articles.columns)



Index(['title', 'source', 'published', 'language', 'quality_score_advanced',
       'entities_total', 'sentence_count', 'readability_score',
       'text_complexity'],
      dtype='object')


In [14]:
df_articles.head()

,title,source,published,language,quality_score_advanced,entities_total,sentence_count,readability_score,text_complexity
0,"""On n'a fait que du Tetris"" : on vous raconte l'histoire secrète de la cérémonie d'ouverture des Jeux de Paris, un an après",https://www.francetvinfo.fr/titres.rss,2025-07-26T05:00:37,fr,0.796034,176,58,0.780172,1.0
1,"Planète 9, comètes interstellaires et migration dans le Système solaire : que peut nous apprendre l'Observatoire Vera-Rubin ?",https://www.futura-sciences.com/rss/actualites.xml,2025-07-24T19:04:44,fr,0.640000,74,19,0.000000,1.0
2,"Paris 2024 : Adidas Arena, Centre aquatique olympique, marina de Marseille... Un an après les Jeux, la promesse de l'héritage des sites pérennes est-elle tenue ?",https://www.francetvinfo.fr/titres.rss,2025-07-26T04:00:02,fr,0.666207,63,29,0.131034,1.0
3,AI Act : un changement de taille attend les géants de l’IA le 2 août prochain : qu’est ce que ça va changer ?,https://www.01net.com/rss/actualites/,2025-07-26T05:45:40,fr,0.643704,31,27,0.018519,1.0
4,"Un an après l'or : blues, bébé ou rebond pour nos champions ?",https://www.france24.com/fr/rss,2025-07-25T14:02:38,fr,0.724615,70,39,0.423077,1.0


In [15]:
DEVICE = "cuda"  # ou "cpu"
FUSION_STRATEGY = "adaptive"
DOMAIN = None
VERBOSE = True
RANDOM_SEED = 42


In [16]:
result = summarize_text_with_ensemble(
    text=raw_text,
    strategy=FUSION_STRATEGY,
    domain=DOMAIN,
    device=DEVICE,
    verbose=VERBOSE,
    random_seed=RANDOM_SEED
)

print("Résumé fusionné :\n", result['ensemble_summary']['summary'])
print("Longueur du résumé :", result['ensemble_summary']['length'], "mots")


INFO:scripts.summarizer_engine:Seed d'aléatoire fixée à 42
INFO:scripts.summarizer_engine:Initialisation des modèles abstractifs...
INFO:models.abstractive_models: Chargement moussaKam/barthez...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
df_summaries = pd.DataFrame(result['individual_summaries'])
display(df_summaries[['model', 'type', 'summary', 'confidence', 'length']])

evaluator = ComprehensiveEvaluator(device=DEVICE, lang="fr")
evaluation = evaluator.evaluate_summary(
    summary=result['ensemble_summary']['summary'],
    source=raw_text
)

main_scores = {
    "ROUGE-1 (f)": evaluation.get('rouge_scores', {}).get('rouge1_f'),
    "BERTScore (f1)": evaluation.get('bert_score', {}).get('bertscore_f1'),
    "Factualité": evaluation.get('factuality', {}).get('factuality_score'),
    "Cohérence": evaluation.get('coherence', {}).get('coherence_score'),
    "Lisibilité": evaluation.get('readability', {}).get('readability_score'),
    "Engagement": evaluation.get('engagement', {}).get('engagement_score'),
    "Densité info": evaluation.get('information_density', {}).get('information_density'),
    "Score composite": evaluation.get('composite_score', {}).get('composite_score')
}
display(pd.DataFrame([main_scores]))


In [ ]:
import matplotlib.pyplot as plt

scores = {k: v for k, v in main_scores.items() if v is not None}
plt.figure(figsize=(9, 4))
plt.bar(scores.keys(), scores.values())
plt.xticks(rotation=45, ha='right')
plt.title("Scores principaux du résumé fusionné")
plt.tight_layout()
plt.show()


In [ ]:
result_file = RESULTS_DIR / f"{result['summary_id']}_summary_and_scores.json"
with open(result_file, "w", encoding="utf-8") as f:
    json.dump({
        "summary": result["ensemble_summary"],
        "individual_summaries": result["individual_summaries"],
        "evaluation": evaluation
    }, f, ensure_ascii=False, indent=2)
print("Résultats exportés dans :", result_file)


In [ ]:
"""human_eval = HumanEvaluationInterface(output_dir=RESULTS_DIR / "human_eval")
task = human_eval.create_evaluation_task(
    summaries=result['individual_summaries'],
    source_text=raw_text,
    task_id=result['summary_id']
)
print("Tâche d'évaluation humaine créée :", task['task_id'])"""


In [ ]:
# Charger un corpus pour traitement batch
BATCH_CSV_PATH = PROCESSED_DIR / "articles_preprocessed_summary.csv"
df_corpus = pd.read_csv(BATCH_CSV_PATH)

# Liste des textes (adapter le nom de colonne selon ta structure)
TEXTS = df_corpus['text'].tolist()
TEXT_IDS = df_corpus['id'].tolist() if 'id' in df_corpus else list(range(len(TEXTS)))

# Stratégies à comparer
FUSION_STRATEGIES = ["adaptive", "confidence_weighted"]


In [ ]:
from tqdm.notebook import tqdm

results_batch = []

for fusion_strategy in FUSION_STRATEGIES:
    print(f"=== Traitement batch : stratégie '{fusion_strategy}' ===")
    for idx, (text_id, text) in enumerate(tqdm(zip(TEXT_IDS, TEXTS), total=len(TEXTS))):
        try:
            summary_result = summarize_text_with_ensemble(
                text=text,
                strategy=fusion_strategy,
                domain=None,
                device=DEVICE,
                verbose=False
            )
            # Évaluation
            evaluator = ComprehensiveEvaluator(device=DEVICE, lang="fr")
            evaluation = evaluator.evaluate_summary(
                summary=summary_result['ensemble_summary']['summary'],
                source=text
            )
            main_scores = {
                "text_id": text_id,
                "fusion_strategy": fusion_strategy,
                "summary": summary_result['ensemble_summary']['summary'],
                "length": summary_result['ensemble_summary']['length'],
                "rouge1_f": evaluation.get('rouge_scores', {}).get('rouge1_f'),
                "bertscore_f1": evaluation.get('bert_score', {}).get('bertscore_f1'),
                "factuality": evaluation.get('factuality', {}).get('factuality_score'),
                "coherence": evaluation.get('coherence', {}).get('coherence_score'),
                "readability": evaluation.get('readability', {}).get('readability_score'),
                "engagement": evaluation.get('engagement', {}).get('engagement_score'),
                "info_density": evaluation.get('information_density', {}).get('information_density'),
                "composite_score": evaluation.get('composite_score', {}).get('composite_score')
            }
            results_batch.append(main_scores)
        except Exception as e:
            print(f"Erreur sur texte {text_id}, stratégie {fusion_strategy} :", e)


In [ ]:
df_batch = pd.DataFrame(results_batch)
display(df_batch.head())




In [ ]:
# Export CSV pour analyse ultérieure
batch_result_path = RESULTS_DIR / "batch_summary_comparison.csv"
df_batch.to_csv(batch_result_path, index=False)
print("Résultats batch exportés :", batch_result_path)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Comparer les scores composites par stratégie
plt.figure(figsize=(10, 5))
sns.boxplot(
    data=df_batch,
    x="fusion_strategy",
    y="composite_score",
    palette="Set2"
)
plt.title("Distribution des scores composites par stratégie de fusion")
plt.ylabel("Score composite")
plt.xlabel("Stratégie de fusion")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=df_batch,
    x="factuality",
    y="coherence",
    hue="fusion_strategy",
    alpha=0.6
)
plt.title("Factualité vs Cohérence selon la stratégie de fusion")
plt.xlabel("Factualité")
plt.ylabel("Cohérence")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Corrélation des métriques principales
metrics = ["rouge1_f", "bertscore_f1", "factuality", "coherence", "readability", "engagement", "info_density", "composite_score"]
corr = df_batch[metrics].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Corrélation entre les différentes métriques de résumé")
plt.tight_layout()
plt.show()


In [ ]:
# Afficher les textes ayant le meilleur score composite pour chaque stratégie
top_n = 3
for strategy in FUSION_STRATEGIES:
    print(f"\nTop {top_n} pour la stratégie {strategy} :")
    display(df_batch[df_batch["fusion_strategy"] == strategy].sort_values("composite_score", ascending=False).head(top_n)[["text_id", "summary", "composite_score"]])


In [ ]:
detailed_result_path = RESULTS_DIR / "batch_summary_comparison_detailed.json"
df_batch.to_json(detailed_result_path, orient="records", force_ascii=False, indent=2)
print("Résultats détaillés exportés :", detailed_result_path)
